## RUN - Nuclear NICD Bleaching

### Notes

- This is basically the same as `RUN - nuclear nicd measurement.ipynb` but without the complication of endpoint measurements

### Prep

In [ ]:
# Imports

import os, csv

import numpy as np
import scipy.ndimage as ndi

from skimage import io
from skimage.filters import threshold_otsu
from skimage.morphology import disk

In [ ]:
# Parameters

img_dir = r'..\Data\Images\HisGFP\bleaching control'
out_dir = r'..\Data\Measurements\HisGFP\bleaching control'

camera_noise = 10    # Max intensity to still be considered "dark current"
min_line_pxls = 100  # Min non-"dark current" pixels per line to not be cropped away

target_slice = 13  # His-GFP stack slice index

In [ ]:
# Find all relevant dirs and files

# Get all file paths
walk = os.walk(img_dir)

# Create a dict for all samples
path_dict = dict()
for w in walk:
    for f in w[-1]:
        if 'His-GFP' in f:
            path_dict[w[0]] = dict()

# All relevant files
for dpath in path_dict:
    for f in os.listdir(dpath):
        if 'His-GFP timeseries' in f:
            path_dict[dpath]['nuc'] = f
        elif 'mCh-NICD timeseries' in f:
            path_dict[dpath]['nicd'] = f
    if len(path_dict[dpath]) != 2:
        raise Exception("Incorrect number of files found for this path!")

# Report
for dpath in path_dict:
    print(dpath)
    for fkey in path_dict[dpath]:
        print('    {:<4}'.format(fkey), '--', path_dict[dpath][fkey])

In [ ]:
# Manually annotate active pulse times
pulse_indices_manual = [0,1,2,3,4,16,17,18,19,20,32,33,34,35,36]
pulse_times_manual = [0,1,2,3,4,15,16,17,18,19,30,31,32,33,34]

### Helper Functions

In [ ]:
def crop_camera(img, ref_img, camera_noise, min_line_pxls):
    
    if ref_img.ndim == 2:
        crop_mask = np.sum(ref_img > camera_noise, axis=1) >= min_line_pxls
    elif ref_img.ndim == 3:
        crop_mask = np.sum(ref_img[0] > camera_noise, axis=1) >= min_line_pxls
    else:
        raise Exception("Unanticipated dimensionality of ref_img:", ref_img.ndim)
        
    img = img[..., crop_mask, :]
    
    return img

### Pipeline Function

In [ ]:
def run_pipeline(dpath, fnames, target_slice, batch_name,
                 camera_noise=10, min_line_pxls=100):
    
    # Report
    print("\nWorking on:", dpath)
    
    # Load data
    data = {}
    for fkey in fnames:
        img = io.imread(os.path.join(dpath, fnames[fkey]))
        data[fkey] = img
    
    # Select relevant data
    if batch_name == 'Continuous':
        data['nuc'] = data['nuc'][::2, target_slice, :, :]
    if batch_name == 'Pulsatile':
        data['nuc'] = data['nuc'][::2, target_slice, :, :]
        data['nicd'] = data['nicd'][pulse_indices_manual, :, :]
        
    # For some samples, there is a mismatch in length of nuc and nicd...
    if data['nicd'].shape[0] != data['nuc'].shape[0]:
        max_len = min([data['nicd'].shape[0], data['nuc'].shape[0]])
        data['nuc']  = data['nuc'][:max_len]
        data['nicd'] = data['nicd'][:max_len]
    
    # Crop top and bottom
    ref_img = data['nuc'].copy()
    print("-- (y,x) cropped from", ref_img[0].shape, end=' ')
    for fkey in data:
        data[fkey] = crop_camera(data[fkey], ref_img, camera_noise, min_line_pxls)
    print("to", data['nuc'][0].shape)
    
    # Report what has been loaded
    print("-- Loaded and prepped arrays have these shapes:")
    for dkey in data:
        print("   ", dkey+':', data[dkey].shape)
    
    # Gaussian smoothing
    data['nuc_smooth'] = np.array([ndi.gaussian_filter(data['nuc'][t], 2) 
                                   for t in range(data['nuc'].shape[0])])
    
    # Otsu thresholding
    threshold = threshold_otsu(data['nuc_smooth'].flatten())
    data['nuc_mask']  = data['nuc_smooth'] > threshold
    
    # Getting whole cells (incl. cytoplasm)
    data['all_mask'] = np.array([ndi.binary_dilation(data['nuc_mask'][t], 
                                                     structure=disk(10, dtype=np.bool))
                                 for t in range(data['nuc'].shape[0])])
    data['cyt_mask'] = data['all_mask'] ^ data['nuc_mask']
    print("-- Completed image processing steps")
    
    # Save resulting segmentation (for QC)
    qc_out = np.array([data['nuc_mask'], data['nuc'], data['nicd']])
    qc_out = np.moveaxis(qc_out, [0,1], [1,0])
    io.imsave(os.path.join(dpath, 'QC_segmentation.tif'), qc_out)
    print("-- Saved segmentations for QC")

    # Get more conservative masks to avoid misattribution
    data['nuc_mask_cons'] = np.array([ndi.binary_erosion(data['nuc_mask'][t], 
                                                         structure=disk(5, dtype=np.bool))
                                      for t in range(data['nuc'].shape[0])])
    data['nuc_mask_dilated'] = np.array([ndi.binary_dilation(data['nuc_mask'][t], 
                                                                structure=disk(3, dtype=np.bool))
                                            for t in range(data['nuc'].shape[0])])
    data['cyt_mask_cons'] = data['all_mask'] ^ data['nuc_mask_dilated']
    
    # Get measurements over time
    m = measurements = dict()
    measurements['nuc_nicd_sum']  = [data['nicd'][t][data['nuc_mask'][t]].sum() 
                                     for t in range(data['nuc'].shape[0])]
    measurements['nuc_nicd_mean'] = [data['nicd'][t][data['nuc_mask_cons'][t]].mean() 
                                     for t in range(data['nuc'].shape[0])]
    measurements['nuc_area']      = data['nuc_mask'].sum(axis=(-1, -2))
    
    measurements['cyt_nicd_sum']  = [data['nicd'][t][data['cyt_mask'][t]].sum() 
                                     for t in range(data['nuc'].shape[0])]
    measurements['cyt_nicd_mean'] = [data['nicd'][t][data['cyt_mask_cons'][t]].mean() 
                                     for t in range(data['nuc'].shape[0])]
    measurements['cyt_area']      = data['cyt_mask'].sum(axis=(-1, -2))

    measurements['all_nicd_sum']  = [data['nicd'][t][data['all_mask'][t]].sum() 
                                     for t in range(data['nuc'].shape[0])]
    measurements['all_nicd_mean'] = [data['nicd'][t][data['all_mask'][t]].mean() 
                                     for t in range(data['nuc'].shape[0])]
    measurements['all_area']      = data['all_mask'].sum(axis=(-1, -2))

    # Convert lists to arrays
    for mkey in measurements:
        if isinstance(m[mkey], list):
            m[mkey] = np.array(m[mkey])
    
    # Prepare for saving
    sample_number = dpath[-1]
    fname_midfix = '_NICDbleach'
    dpath_out = dpath.replace(os.path.relpath(img_dir), os.path.relpath(out_dir))
    fpath_out = os.path.join(dpath_out, sample_number + '_'+batch_name + fname_midfix+'.tsv')
    if not os.path.isdir(dpath_out):
        os.makedirs(dpath_out)
    
    # Save results
    with open(fpath_out, 'w', newline='') as outfile:
        writer = csv.writer(outfile, delimiter='\t')
        measurement_keys = [mkey for mkey in measurements.keys()]
        writer.writerow(['time_step'] + measurement_keys)
        for t in range(len(measurements['nuc_nicd_sum'])):
            if batch_name == 'Continuous':
                data = [t] + [m[mkey][t] for mkey in measurement_keys]
            if batch_name == 'Pulsatile':
                data = [pulse_times_manual[t]] + [m[mkey][t] for mkey in measurement_keys]
            writer.writerow(data)

    print("-- Finished!")

### Execution Loop

In [ ]:
for dpath in path_dict:
    
    if 'bleaching_continuous' in dpath:
        batch_name = 'Continuous'
    elif 'bleaching_pulsatile' in dpath:
        batch_name = 'Pulsatile'
        
    run_pipeline(dpath, path_dict[dpath], target_slice, batch_name,
                 camera_noise=camera_noise, min_line_pxls=min_line_pxls)